# Assignment 4

In [232]:
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #We need this to split the data
from sklearn.preprocessing import normalize #get the function needed to normalize our data.
from sklearn.neighbors import KNeighborsClassifier #the object class we need

In [233]:
df = pd.read_csv('CompleteDataset.csv')
df = df.dropna() #first get rid of rows with empty cells
df.head()

/Users/julka/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,...,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,1,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,2,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,...,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,3,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,...,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
5,5,R. Lewandowski,28,https://cdn.sofifa.org/48/18/players/188545.png,Poland,https://cdn.sofifa.org/flags/37.png,91,91,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,...,58.0,57.0,78.0,62.0,87.0,82.0,88.0,84.0,61.0,88.0


In [234]:
def split(x):
    return x.split()[0]

df["Preferred Position"] = df["Preferred Positions"].apply(split)

In [235]:
df['Preferred Position'].value_counts()

CB     2660
ST     2261
CM     1953
CDM    1372
RM     1327
LM     1312
LB     1292
RB     1186
CAM    1109
RW      426
LW      422
CF      199
LWB     119
RWB     108
Name: Preferred Position, dtype: int64

In [244]:
df_sel = df[['Long passing', 'Stamina', 'Interceptions', 'Agility', 'Free kick accuracy', 'Finishing']]

# the data frame has a lot of equations as strings in values for example 80+1, here i try to avoid this problem

def correct(x):
    if isinstance(x, str):
        if('-' in x):
            return float(x.split('-')[0]) - float(x.split('-')[1])
        elif('+' in x):
            return float(x.split('+')[0]) + float(x.split('+')[1])
        else:
            return float(x)
        
    else:
        return float(x)
        

for column in df_sel.columns:
    df_sel[column] = df_sel[column].apply(correct)

/Users/julka/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [245]:
X = df_sel #create the X matrix

try:
    X = normalize(X) #normalize the matrix to put everything on the same scale
except:
    pass

y = df['Preferred Position'] #create the y-variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) #split the data, store it into different variables

In [246]:
knn = KNeighborsClassifier(n_neighbors=8) #create a KNN-classifier with 5 neighbors (default)
knn = knn.fit(X_train, y_train) #this fits the k-nearest neigbor model with the train data
knn.score(X_test, y_test) #calculate the fit on the test data

0.3837849280270957

In [247]:
from sklearn.metrics import confusion_matrix

y_test_pred = knn.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_test_pred) #creates a "confusion matrix"
cm

array([[ 81,   1,  37,   3,  91,   6,  60,   8,   0,   3,  34,   3,   0,
         40],
       [  5, 577,  45,   0,  56,  32,   3,   0,   0,  29,   1,   0,   0,
          1],
       [ 32,  97, 114,   0, 105,  31,  11,   1,   0,  18,   5,   0,   0,
          0],
       [ 14,   1,   1,   0,   6,   0,   7,   2,   0,   0,   8,   0,   0,
         22],
       [ 56,  68, 132,   1, 215,  28,  26,   2,   0,  13,  20,   0,   0,
          5],
       [  8, 140,  52,   0,  47, 101,  12,   0,   2,  45,   2,   0,   0,
          1],
       [ 67,  22,  23,   3,  58,  28,  70,   5,   0,  14,  34,   5,   0,
         65],
       [ 19,   0,   1,   5,   5,   2,  30,   4,   0,   0,  13,   2,   0,
         34],
       [  4,   5,   7,   0,   2,  10,   1,   0,   0,   1,   1,   0,   0,
          0],
       [  6, 124,  42,   0,  37,  62,  12,   1,   0,  46,   5,   1,   0,
          2],
       [ 74,  17,  22,   2,  68,  20,  72,   9,   0,   7,  50,   6,   0,
         53],
       [ 21,   2,   5,   2,  10,   1,  24, 

In [249]:
#In order to read it easily , let's make a dataframe out of it, and add labels to it.
conf_matrix = pd.DataFrame(cm, index=['CB', 'ST', 'CM', 'CDM', 'RM', 
                                      'LM', 'LB', 'RB', 'CAM', 'RW', 'LW', 'CF', 'LWB', 'RWB'],
                          columns = ['CB_p', 'ST_p', 'CM_p', 'CDM_p', 'RM_p', 
                                      'LM_p', 'LB_p', 'RB_p', 'CAM_p', 'RW_p', 'LW_p', 'CF_p', 'LWB_p', 'RWB_p'])
conf_matrix

,CB_p,ST_p,CM_p,CDM_p,RM_p,LM_p,LB_p,RB_p,CAM_p,RW_p,LW_p,CF_p,LWB_p,RWB_p
CB,81,1,37,3,91,6,60,8,0,3,34,3,0,40
ST,5,577,45,0,56,32,3,0,0,29,1,0,0,1
CM,32,97,114,0,105,31,11,1,0,18,5,0,0,0
CDM,14,1,1,0,6,0,7,2,0,0,8,0,0,22
RM,56,68,132,1,215,28,26,2,0,13,20,0,0,5
LM,8,140,52,0,47,101,12,0,2,45,2,0,0,1
LB,67,22,23,3,58,28,70,5,0,14,34,5,0,65
RB,19,0,1,5,5,2,30,4,0,0,13,2,0,34
CAM,4,5,7,0,2,10,1,0,0,1,1,0,0,0
RW,6,124,42,0,37,62,12,1,0,46,5,1,0,2
